In [ ]:
import pandas as pd
import json
import plotly.express as px
from urllib.request import urlopen

#Load map of Amsterdam
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

In [ ]:
# --- Sensorlocaties ---
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"].astype(str).str.strip(),  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"].astype(str).str.strip(), errors="coerce")

#Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

#----Predicted data ----
cs = pd.read_csv("predicted_sensor_values_3min.csv", parse_dates=["timestamp"]) #use real time 
cs = cs.sort_values("timestamp") #sort on time

In [ ]:
#only predicted, remove actual sensors  




#only actual values 

In [ ]:
#angle toevoegen 